# Example of how to "get started" with the CMG project

Import modules and configure the notebook.

In [ ]:
# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt

# This is my own script (it is an interface to the pybullet simulator)
import ae353_cmg

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_cmg)

Create an instance of the robot simulator. This will open a window in which you should see the platform with a single-gimbal control moment gyroscope.

There are three optional parameters:

* `damping` is the coefficient of viscouse friction at each joint (the default value is `0.`)
* `dt` is the length of each time step in the simulator (the default value is `0.001`)
* `display` is a flag that says whether or not to open the GUI (the default value is `True`)

The `display` parameter, in particular, is likely to be of use to you. If you set `display=True`, then you will run the simulator in real-time with a GUI. If you set `display=False`, then you will run the simulator as fast as possible (faster than real-time) without a GUI. This is convenient when all you want is the data.

**NOTE:** it is still necessary to reset the kernel (see the "Kernel" menu above) before creating a new instance of the robot simulator, even when running without a GUI - if you don't, then you'll notice that simulation slows way down.

In [ ]:
robot = ae353_cmg.RobotSimulator(damping=0., dt=0.001, display=True)

Apply PD control to keep the gimbal angle at a desired value:

In [ ]:
class RobotController:
    def __init__(self, dt=0.001, q_2_des=0.):
        self.dt = dt
        
        # desired gimbal angle
        self.q_2_des = q_2_des
        
        # PD gains
        self.kp = 5.
        self.kd = 1.
    
    def run(self, q_1, v_1, q_2, v_2, q_3, v_3):
        tau_2 = - self.kp * (q_2 - self.q_2_des) - self.kd * (v_2 - 0.)
#         tau_2 = 1.
        tau_3 = 0.
        return tau_2, tau_3

controller = RobotController(dt=robot.dt, q_2_des=0.855211)

Apply linear state feedback (equivalent to PD control) to keep the gimbal angle at a desired value:

In [ ]:
class RobotController:
    def __init__(self, dt=0.001, q_2_des=0.):
        self.dt = dt
        
        # desired gimbal angle
        self.q_2_des = q_2_des
        
        # state feedback gains
        self.K = np.array([[5., 1.]])
        
        # equilibrium point
        self.q_2_e = self.q_2_des
        self.v_2_e = 0.
        self.tau_2_e = 0.
    
    def run(self, q_1, v_1, q_2, v_2, q_3, v_3):
        # state
        x = np.array([[q_2 - self.q_2_e], [v_2 - self.v_2_e]])
        
        # input
        u = - self.K @ x
        
        # actuator commands
        tau_2 = u[0, 0] + self.tau_2_e
        tau_3 = 0.
        return tau_2, tau_3

controller = RobotController(dt=robot.dt, q_2_des=-1.)

Run the simulation. It is a loop. At each iteration, we:
* get sensor measurements
* choose actuator commands
* go forward one time step

We also log data so that we can plot it later, if we want.

In [ ]:
# Restore the simulation to its initial state
robot.reset(rotor_rpm=100.)

# Choose how long we want to run the simulation, and
# compute the corresponding number of time steps
run_time = 50.
num_steps = int(run_time/robot.dt)

# Create a dictionary in which to store results
data = {
    't': np.empty(num_steps, dtype=float),
    'q_1': np.empty(num_steps, dtype=float),
    'v_1': np.empty(num_steps, dtype=float),
    'q_2': np.empty(num_steps, dtype=float),
    'v_2': np.empty(num_steps, dtype=float),
    'q_3': np.empty(num_steps, dtype=float),
    'v_3': np.empty(num_steps, dtype=float),
    'tau_2': np.empty(num_steps, dtype=float),
    'tau_3': np.empty(num_steps, dtype=float),
}

# Run the simulation loop
start_time = time.time()
for step in range(num_steps):
    # Get the current time
    t = robot.dt * step
    
    # Get the sensor measurements
    q_1, v_1, q_2, v_2, q_3, v_3 = robot.get_sensor_measurements()
    
    # Choose the actuator command (by running the controller)
    tau_2, tau_3 = controller.run(q_1, v_1, q_2, v_2, q_3, v_3)
    
    # Log the data from this time step
    data['t'][step] = t
    data['q_1'][step] = q_1
    data['v_1'][step] = v_1
    data['q_2'][step] = q_2
    data['v_2'][step] = v_2
    data['q_3'][step] = q_3
    data['v_3'][step] = v_3
    data['tau_2'][step] = tau_2
    data['tau_3'][step] = tau_3
    
    # Send the actuator commands to robot and go forward one time
    # step (this is where the actual simulation happens)
    robot.set_actuator_commands(tau_2, tau_3)
    robot.step(t=(start_time + (robot.dt * (step + 1))))

An example of how to plot results. This is just an example!

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(data['t'], data['q_2'], label='gimbal angle (rad)', linewidth=4)
plt.plot(data['t'], np.ones_like(data['t']) * controller.q_2_des, '--', label='desired gimbal angle (rad)', linewidth=4)
plt.plot(data['t'], data['v_2'], ':', label='gimbal angular velocity (rad/s)', linewidth=3)
plt.legend(fontsize=18)
plt.grid()
plt.tick_params(labelsize=16)
plt.xlabel('time (s)', fontsize=16)